In [ ]:
!pip install mediapipe
!pip install opencv-python

In [ ]:
import cv2
import mediapipe as mp
import math
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def calculate_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# 辨識手勢
def recognize_gesture(hand_landmarks):
    landmarks = hand_landmarks.landmark
    points = {
        name: [landmarks[getattr(mp_hands.HandLandmark, name)].x, 
               landmarks[getattr(mp_hands.HandLandmark, name)].y]
        for name in ['WRIST', 'THUMB_TIP', 'INDEX_FINGER_TIP', 'MIDDLE_FINGER_TIP', 
                     'RING_FINGER_TIP', 'PINKY_TIP', 'THUMB_IP', 'THUMB_CMC']
    }

    # 計算手掌中心
    palm_center = np.mean(
        [points['WRIST'], points['THUMB_CMC'], points['INDEX_FINGER_TIP'], points['PINKY_TIP']], axis=0
    )

    # 1. Fist: 手指尖接近手掌中心
    if all(calculate_distance(points[f"{finger}_TIP"], palm_center) < 0.1 for finger in ['INDEX_FINGER', 'MIDDLE_FINGER', 'RING_FINGER', 'PINKY']):
        return "Fist"

    # 2. OK Sign: 拇指和食指尖接近，其他手指伸直
    if (
        calculate_distance(points['THUMB_TIP'], points['INDEX_FINGER_TIP']) < 0.1 and
        all(calculate_distance(points[f"{finger}_TIP"], palm_center) > 0.2 for finger in ['MIDDLE_FINGER', 'RING_FINGER', 'PINKY'])
    ):
        return "OK Sign"

    # 3. Open Palm: 所有手指遠離手掌
    if all(calculate_distance(points[f"{finger}_TIP"], palm_center) > 0.2 for finger in ['INDEX_FINGER', 'MIDDLE_FINGER', 'RING_FINGER', 'PINKY']):
        return "Open Palm"

    return "Unknown Gesture"


cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("無法讀取攝像頭畫面")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        gesture = "No Hand Detected"

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                gesture = recognize_gesture(hand_landmarks)


        cv2.putText(frame, f"Gesture: {gesture}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow("Hand Gesture Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()